In [290]:
from sympy import *
import numpy as np
import scipy as sp
from scipy.linalg import lu
from scipy.sparse.linalg import spilu
import math
import sympy 
import itertools
import sys
import time
import multiprocessing

from __future__ import print_function


from copy import deepcopy
import time
import fractions
import timeit

# print function

In [291]:
def print_row(row,symbols,nvar,cal=False):
    # nvar  
    plus_flag = False
    
    for ele in row:
        mul_flag = False
        for i in range(nvar+1):
            if i==0:
                if plus_flag and ele[i]>0 and :
                    print('+',end ='')
                if ele[i] != 1:
                    if cal:
                        print(ele[i],end="")
                    else:
                        print("%2.2f"%ele[i],end="")
                    mul_flag = True
                plus_flag = True
                continue
            if ele[i] != 0:
                if mul_flag:
                    print('*',end = "")
                if cal :
                    mul_flag = True
                print(symbols[i-1],end = "")
                if ele[i] > 1:
                    if cal:
                        print("**",end="")
                    else:
                        print("^",end="")
                    print(ele[i],end="")
        plus_flag = True
    if cal:
        print(",")
    else:
        print("")
            
def print_all(M,symbols,nvar,cal=False):
    for row in M:
        if len(row)!=0:
            print_row(row,symbols,nvar,cal)

# formation of the coefficient matri for multi-variable

In [292]:
def matC_sparse(polynomials, mdeg,nvar=2):
    row = []
    col =[]
    ele = []
    
    rows = len(polynomials)
    cols = mdeg**nvar
    
    for i in range(rows):
        for monomial in polynomials[i]:
            s = cols-1
            for j in range(nvar):
                s -= monomial[j+1]*mdeg**(nvar-1-j)
            row.append(i)
            col.append(s)
            ele.append(1.0*monomial[0])
    M = sp.sparse.csr_matrix((ele, (row, col)), shape=(rows, cols))
    return M


# Gaussian Elimination

In [293]:
def GEPP_sparse(C):    
    M1 = C.copy()
    M1 = M1.tocsc()
    n = M1.shape[0]  ## number of rows
    m = M1.shape[1]  ## number of columns

    currentrow = 0
    for i in range(m):
        if M1[currentrow:,i].nnz==0:
            pass  ## do nothing if we have a column of zeros
        else:
            if currentrow < (n-1):
                    #print(i)
                    ## first find index of (abs) maximum value
                non_zero_row = M1[currentrow:n,i]
                sub_index = np.argmax(abs(sp.sparse.find(non_zero_row)[2]))
                index = sp.sparse.find(non_zero_row)[0][sub_index]+currentrow  #### n\n-1 
                    #print(M[currentrow:n,i])
                    #print(index)
                    #print("index is", index,"\n")
                # swap rows
                a_idx = np.where(M1.indices == index)
                b_idx = np.where(M1.indices == currentrow)
                M1.indices[a_idx] = currentrow
                M1.indices[b_idx] = index    
                          ####
                #print("the matrix is now \n",M,"\n")
                #print(M)

                    ## running the elimination step
                for j in range(currentrow+1,n):
                    if M1[j,i]!=0:
                            #print("currentrow column entry is", M[currentrow,i],"\n")
                        multiplier = M1[j,i]/M1[currentrow,i]
                            #print("multiplier is", multiplier,"\n")
                        M1[j,i:] = M1[j,i:] - multiplier*M1[currentrow,i:]
                    else:
                        pass
                    #print("the matrix is after elimination \n",M,"\n")

                    ## incrementing row so the algorithm stops when we have a triangular system
    
                currentrow = currentrow + 1
    
    M1=M1.tocsr()
    
    for j in range(n):
        if M1[j,:].nnz==0:
            continue  ## do nothing if we have a row of zeros
        else:
       

            indptr = M1.indptr
            a = indptr[j]
            b = indptr[j+1]
            M1.data[a:b] = M1.data[a:b]/M1.data[a] 
      

    return M1.tocsr()


# Buchberger's algorithm for multi-variable

In [294]:
## sparse

# input the start th leading
def leading_sparse(row,start=0):
    if row.nnz==0:
        return -1
    if start >= row.nnz:
        return -1
        
        print("leading term overflow")
        print(row)
        print(start)
        sys.exit()
    
    return row.indices[start]
    # return sp.sparse.find(row)[1][start]
    

def leading_term_sparse(arg,mdeg,nvar):
    # return leading term without coefficient
    
    result = np.zeros(nvar)
    a = arg
    b = 0
    for i in range(nvar):
        div = mdeg**(nvar-1-i)
        b = a - a/div*div
        a = a/div
        result[i]=mdeg-1-a
        a = b
    return result

def check_multiple_sparse(lead_1,lead_2,nvar):
    # check lead_1 is a multiple of lead_2
    for i in range(nvar):
        if lead_1[i]<lead_2[i]:
            return False
    return True

def shift_col_sparse(lcm,deg_i,mdeg,nvar):
    a = lcm[:-1]-deg_i
    shift_column = 0
    for i in range(nvar):
        shift_column += mdeg**(nvar-1-i)*a[i]
    return int(shift_column)

# input sparse matirx
def shift_sparse(lcm,deg_i,mdeg,nvar,row=-1,leading=-1,S=(-1,-1)):
    row = row.tocsr()
    row_copy = deepcopy(row)
    shift_column = shift_col_sparse(lcm,deg_i,mdeg,nvar)
    if leading-shift_column < 0:
        print(S,leading,shift_column)
        print(row)
        print("degree overflow")
        sys.exit()
    row_copy.indices[:] -= shift_column

    return row_copy

def get_lcm_sparse(a,b): 
    return abs(a * b) / fractions.gcd(a,b) if a and b else 0

def check_multiple_sparse(lead_1,lead_2,nvar):
    # check lead_1 is a multiple of lead_2
    for i in range(nvar):
        if lead_1[i]<lead_2[i]:
            return False
    return True

def remainder_sparse(M,new_row,mdeg, nvar):
    M = M.tocsr()
    flag = True
    m = M.shape[0]
    while flag:
        flag = False
        new_lead = leading_sparse(new_row)
        if new_lead==-1:
            break
        new_lead_term = leading_term_sparse(new_lead,mdeg,nvar)
        for i in range(m):
            row_lead = leading_sparse(M[i])
            row_lead_term = leading_term_sparse(row_lead,mdeg,nvar)
            if check_multiple_sparse(new_lead_term,row_lead_term,nvar):
                #print(i)
                lcm = np.hstack((new_lead_term,[1]))
                old_shifted = shift_col_sparse(lcm,row_lead_term,mdeg,nvar)
                coeff = new_row[0,new_lead]
                
                current_row = deepcopy(M[i])
                #print(current_row)
                current_row.indices[:] -= old_shifted
                #print(current_row)
                
                new_row -= coeff * current_row
                flag = True
                break
    return new_row

def Buchberger_sparse(M, mdeg, nvar, S):
    # check leading term
    a = M[S[0]]
    b = M[S[1]]
    
    arg_a = leading_sparse(a)
    arg_b = leading_sparse(b)
    
    if arg_a==-1 or arg_b==-1:
        return 0,M
        
        print("0 row occur")
        print(S)
        print(M)
        sys.exit()
    
    deg_a = leading_term_sparse(arg_a,mdeg,nvar)
    deg_b = leading_term_sparse(arg_b,mdeg,nvar)
    
    
    lcm = np.zeros(nvar+1)
    for i in range(nvar):
        lcm[i] = max(deg_a[i],deg_b[i])
    lcm[nvar] = get_lcm_sparse(a[0,arg_a],b[0,arg_b])
    

    a_shifted = shift_sparse(lcm,deg_a,mdeg,nvar,a,arg_a,S)
    b_shifted = shift_sparse(lcm,deg_b,mdeg,nvar,b,arg_b,S)
    
    """   
    if a[0,arg_a]!=1 or b[0,arg_b]!=1:
        print("Normalization in Gaussian Elimination Failed")
        print(M)
        sys.exit()
    """
    if a[0,arg_a]> b[0,arg_b]:
        new_row = a_shifted - (a[0,arg_a]/ b[0,arg_b])*b_shifted
    else:
        new_row = (b[0,arg_b]/a[0,arg_a])*a_shifted - b_shifted
    
    new_row = remainder_sparse(M,new_row,mdeg, nvar)
    

    if new_row.nnz == 0:
        return 0,M
    
    # normalize new column
    arg_c = leading_sparse(new_row)
    
    if new_row[0,arg_c]!=1:
        new_row.data[:] /= new_row[0,arg_c]
    M = sp.sparse.vstack([M,new_row])
    return 1,M



# Reduced 

In [344]:
def leading_sparse(row,start=0):
    if row.nnz==0:
        return -1
    if start >= row.nnz:
        return -1
        
        print("leading term overflow")
        print(row)
        print(start)
        sys.exit()
    
    return row.indices[start]
    # return sp.sparse.find(row)[1][start]
    

def leading_term_sparse(arg,mdeg,nvar):
    # return leading term without coefficient
    
    result = np.zeros(nvar)
    a = arg
    b = 0
    for i in range(nvar):
        div = mdeg**(nvar-1-i)
        b = a - a/div*div
        a = a/div
        result[i]=mdeg-1-a
        a = b
    return result

def check_multiple_sparse(lead_1,lead_2,nvar):
    # check lead_1 is a multiple of lead_2
    for i in range(nvar):
        if lead_1[i]<lead_2[i]:
            return False
    return True

def shift_col_sparse(lcm,deg_i,mdeg,nvar):
    a = lcm[:-1]-deg_i
    shift_column = 0
    for i in range(nvar):
        shift_column += mdeg**(nvar-1-i)*a[i]
    return int(shift_column)

# input sparse matirx
def shift_sparse(lcm,deg_i,mdeg,nvar,row=-1,leading=-1,S=(-1,-1)):
    row = row.tocsr()
    row_copy = deepcopy(row)
    shift_column = shift_col_sparse(lcm,deg_i,mdeg,nvar)
    if leading-shift_column < 0:
        print(S,leading,shift_column)
        print(row)
        print("degree overflow")
        sys.exit()
    row_copy.indices[:] -= shift_column

    return row_copy




def find_largest(lead_v):
    # l is the largest
    l = -1
    arg = -1
    
    m = lead_v.shape[0]
    
    for row in range(m):
        if l == -1:
            l = lead_v[row,-1]
            arg = row
            continue
        if lead_v[row,-1] != -1 and lead_v[row,-1]<l:
            l = lead_v[row,-1]
            arg = row
    if l == -1:
        return -1
    return arg

def renew_lead_v(M,mdeg,nvar,lead_v,row,starts,tol=10**-4):
    m = M.shape[0]
    
    lead_v[row,-1] = leading_sparse(M[row],starts[row])
    while abs(M[row,lead_v[row,-1]]) < tol:
        
        M[row,lead_v[row,-1]]=0
        M.eliminate_zeros()
        lead_v[row,-1] = leading_sparse(M[row],starts[row])

        if lead_v[row,-1] == -1:
            return M,lead_v    
    if lead_v[row,-1] == -1:
        return M,lead_v   
    monomial = leading_term_sparse(lead_v[row,-1],mdeg,nvar)
    for j in range(nvar):
        lead_v[row,j] = monomial[j]
    return M,lead_v
    
def reduced_form(M,mdeg,nvar,tol):
    m,n = M.shape
    
    # the vector store the leading term
    lead_v = np.zeros((m,nvar+1),dtype = int)
    
    # vector stores the i the leading term
    starts = np.zeros(m,dtype = int)
    
    # initialization leading_vector
    for row in range(m):
        M,lead_v = renew_lead_v(M,mdeg,nvar,lead_v,row,starts,tol)
    
    large_row = find_largest(lead_v)
    while large_row != -1:
        flag = False
        for row in range(m):
            if row != large_row and starts[row] == 0:
                lead_arg = leading_sparse(M[row])
                lead_term = leading_term_sparse(lead_arg,mdeg,nvar)
                if check_multiple_sparse(lead_v[large_row,:-1],lead_term,nvar):

                    shift_col = shift_col_sparse(lead_v[large_row],lead_term,mdeg,nvar)
                    indices = deepcopy(M[row].indices) - shift_col
                    indptr = np.zeros(m+1,dtype = int)
                    for i in range(large_row+1,m+1):
                        indptr[i] = indices.shape[0]
                    data = deepcopy(M[row].data)
                    M_subtrac = sp.sparse.csr_matrix((data, indices, indptr), shape=(m, n))
                    c = M[large_row,lead_v[large_row,-1]]/M[row,lead_v[row,-1]]
                    M -= c*M_subtrac
                    flag = True
                    break
        if flag == False:
            starts[large_row]+=1

        M,lead_v=renew_lead_v(M,mdeg,nvar,lead_v,large_row,starts,tol)
        large_row = find_largest(lead_v)
    
    return M
    #print_all(matrix_to_array_sparse(M,mdeg,nvar),['A0','x0','A1','x1'],nvar)

    

In [352]:
p1= [(1,1,0,0,0),(1,0,0,1,0),(-2,0,0,0,0)]
p2= [(1,1,1,0,0),(1,0,0,1,1)]
p3= [(1,1,2,0,0),(1,0,0,1,2),(-2.0/3,0,0,0,0)]
p4= [(1,1,3,0,0),(1,0,0,1,3)]
polynomials = [p1,p2,p3,p4]
mdeg = 20
nvar = 4
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol = 10**-17)
#M.toarray()
print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),['A0','x0','A1','x1'],nvar)

[[1.0, 1.0, 0.0, 0.0, 0.0], [-1.0, 0.0, 0.0, 0.0, 0.0]]
[]
[[1.0, 0.0, 0.0, 0.0, 2.0], [-0.33333333333333331, 0.0, 0.0, 0.0, 0.0]]
[]
[[1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 1.0]]
[[1.0, 0.0, 0.0, 1.0, 0.0], [-3.0, 0.0, 0.0, 0.0, 2.0]]
A0-1.00
x1^2.0-0.33
x0+x1
A1-3.00*x1^2.0


# Combine them together

In [318]:
def delete_row_csr(mat, i):
    if not isinstance(mat, sp.sparse.csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    n = mat.indptr[i+1] - mat.indptr[i]
    if n > 0:
        mat.data[mat.indptr[i]:-n] = mat.data[mat.indptr[i+1]:]
        mat.data = mat.data[:-n]
        mat.indices[mat.indptr[i]:-n] = mat.indices[mat.indptr[i+1]:]
        mat.indices = mat.indices[:-n]
    mat.indptr[i:-1] = mat.indptr[i+1:]
    mat.indptr[i:] -= n
    mat.indptr = mat.indptr[:-1]
    mat._shape = (mat._shape[0]-1, mat._shape[1])

def trun_zero_sparse(M):
    while M[-1,:].nnz==0:
        M = M[:-1,:]
    return M

def normalize_leading_sparse(M):
    m = M.shape[0]
    for i in range(m):
        lead = leading_sparse(M[i])
        if lead == -1:
            #delete_row_csr(M, i)
            continue
        indptr = M.indptr
        a = indptr[i]
        b = indptr[i+1]
        M.data[a:b] = M.data[a:b]/M.data[a]
    return M

In [372]:
def run_sparse(M, mdeg, nvar,tol = 10**-14,details = False,cal = False):
    #M = GEPP_sparse(M)
    
    max_row = M.shape[0]-1
    current_row = 0
    flag = -1
    M=normalize_leading_sparse(M)
    while current_row != max_row:
        #print(current_row,max_row)
        current_row +=1
        #M = GEPP(M)
        #print(M)
        for i in range(current_row):
            M=normalize_leading_sparse(M)
            flag,M = Buchberger_sparse(M, mdeg, nvar, [current_row,i])
            if flag == 1:
                M=reduced_form(M,mdeg,nvar,tol)
                if details:
                    print_all(matrix_to_array_sparse(M,mdeg,nvar),GQ_sym(nvar/2),nvar,cal)
                    print("")
        #time.sleep(1)
        #M = GEPP(M)
        #M = trun_zero(M)
        M=normalize_leading_sparse(M)
        #print(M)
        max_row = M.shape[0]-1
    return M


In [373]:
def matrix_to_array_sparse(M,mdeg,nvar):
    array = []
    m = M.shape[0]
    for i in range(m):
        array.append([])
        index = 0
        nnz = M[i].nnz
        for j in range(nnz):
            index = leading_sparse(M[i],j)
            L_T = leading_term_sparse(index,mdeg,nvar)
            add = [M[i,index]]
            add.extend(L_T)
            array[i].append(add)

    return array

def print_array_sparse(array):
    for p in array:
        print(p)

In [374]:
print_array_sparse(GQ(3))
print_all(GQ(3),GQ_sym(3),6,False)

[[1, 1, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0], [-2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[[1, 1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, 0], [1, 0, 0, 0, 0, 1, 1]]
[[1, 1, 2, 0, 0, 0, 0], [1, 0, 0, 1, 2, 0, 0], [1, 0, 0, 0, 0, 1, 2], [-0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[[1, 1, 3, 0, 0, 0, 0], [1, 0, 0, 1, 3, 0, 0], [1, 0, 0, 0, 0, 1, 3]]
[[1, 1, 4, 0, 0, 0, 0], [1, 0, 0, 1, 4, 0, 0], [1, 0, 0, 0, 0, 1, 4], [-0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[[1, 1, 5, 0, 0, 0, 0], [1, 0, 0, 1, 5, 0, 0], [1, 0, 0, 0, 0, 1, 5]]
A0+A1+A2-2.00
A0x0+A1x1+A2x2
A0x0^2+A1x1^2+A2x2^2-0.67
A0x0^3+A1x1^3+A2x2^3
A0x0^4+A1x1^4+A2x2^4-0.40
A0x0^5+A1x1^5+A2x2^5


In [383]:
n = 4
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = True,cal=True)
#M.toarray()
print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),GQ_sym(n),nvar,cal = True)

A0+A1+A2+A3-2.0,
x0*x1*A2+x0*x1*A3-2.0*x0*x1-1.0*x0*A2*x2-1.0*x0*A3*x3-1.0*x1*A2*x2-1.0*x1*A3*x3+A2*x2**2.0+A3*x3**2.0-0.666666666667,
x0**2.0*x1*A2+x0**2.0*x1*A3-2.0*x0**2.0*x1-1.0*x0**2.0*A2*x2-1.0*x0**2.0*A3*x3-1.0*x1*A2*x2**2.0-1.0*x1*A3*x3**2.0+0.666666666667*x1+A2*x2**3.0+A3*x3**3.0,
x0**3.0*x1*A2+x0**3.0*x1*A3-2.0*x0**3.0*x1-1.0*x0**3.0*A2*x2-1.0*x0**3.0*A3*x3-1.0*x1*A2*x2**3.0-1.0*x1*A3*x3**3.0+A2*x2**4.0+A3*x3**4.0-0.4,
x0**4.0*x1*A2+x0**4.0*x1*A3-2.0*x0**4.0*x1-1.0*x0**4.0*A2*x2-1.0*x0**4.0*A3*x3-1.0*x1*A2*x2**4.0-1.0*x1*A3*x3**4.0+0.4*x1+A2*x2**5.0+A3*x3**5.0,
x0**5.0*x1*A2+x0**5.0*x1*A3-2.0*x0**5.0*x1-1.0*x0**5.0*A2*x2-1.0*x0**5.0*A3*x3-1.0*x1*A2*x2**5.0-1.0*x1*A3*x3**5.0+A2*x2**6.0+A3*x3**6.0-0.285714285714,
x0**6.0*x1*A2+x0**6.0*x1*A3-2.0*x0**6.0*x1-1.0*x0**6.0*A2*x2-1.0*x0**6.0*A3*x3-1.0*x1*A2*x2**6.0-1.0*x1*A3*x3**6.0+0.285714285714*x1+A2*x2**7.0+A3*x3**7.0,
x0*A1+x0*A2+x0*A3-2.0*x0-1.0*A1*x1-1.0*A2*x2-1.0*A3*x3,

A0+A1+A2+A3-2.0,
x0*x1*A2+x0*x1*A3-2.0*x0*x1-1.0*x0*A2*x

In [379]:
A0+A1-0.486111111111*x2**2.0,
x2**3.0+1.0*x2**2.0*x3+1.0*x2*x3**2.0-0.857142857143*x2,
x1**2.0+x1*x2-38.0*x2**2.0*x3**2.0+6.42013888889*x2**2.0+11.0*x2*x3,
A2*x2-0.510308641975*x2**3.0-0.996419753086*x2**2.0*x3-0.996419753086*x2*x3**2.0+0.145740740741*x2,
A1*x2**2.0*x3+0.2*A1*x2*x3**2.0-1.22857142857*A1*x2+1.37142857143*A1*x3-1.62114108801*x1*A2-0.274409471391*x1*x2**2.0+1.00415622389*x1+4.7198589132e+16*A2**2.0*x3-2.97896369151e+16*A2*x2+9.30870017657e+16*x2**3.0+1.30511833872e+17*x2**2.0*x3+1.07568075266e+17*x2*x3**2.0-7.11002145561e+16*x2,
A1*x1-0.3125*A1*x2**2.0*x3+0.708333333333*A1*x2*x3**2.0+0.896428571429*A1*x2-0.0285714285714*A1*x3+0.887734040985*x1*A2+0.165331985214*x1*x2**2.0-0.9596321533*x1-7.32204145268e+16*A2**2.0*x3+6.95682474696e+15*A2*x2-2.8267872854e+16*x2**3.0-6.72429197232e+16*x2**2.0*x3-3.16496626616e+16*x2*x3**2.0+2.22005314189e+16*x2,
x0+x1+0.714285714286*A2*x2+1.03549382716*x2**3.0+0.688271604938*x2**2.0*x3+0.688271604938*x2*x3**2.0-0.0958994708995*x2,
x1*x3+2512.0*x2**2.0*x3**2.0-229.8*x2**2.0-833.142857143*x2*x3-1.86820604367e+16,
A3,
A2+0.486111111111*x2**2.0+3.0041005291

Q =[A0-0.555555555556,
A2-0.888888888889,
x1**2.0-0.6,
x0+x1,
x2,
A1-0.925925925926*x1**2.0]
print(Q)

[-4.4442227675745016e-13, -1.1113332476497817e-13, 1.1102230246251565e-16, 0.0, 0.0, -4.451994328746878e-14]


# Testing

## Generate Gauissian Quaduature 

In [332]:
# nomal Gaussian Quaduature
def GQ(n=2):
    result = []
    for i in range(2*n):
        p = []
        for j in range(n):
            p_s = np.zeros(2*n+1,dtype = int)
            p_s[0] = 1
            p_s[2*j+1] = 1
            p_s[2*j+2] = i
            p.append(p_s.tolist())
        if i-i/2*2 == 0:
            p_s = np.zeros(2*n+1)
            p_s[0] = -2./(i+1)
            p.append(p_s.tolist())
        result.append(p)
    return result

def GQ_sym(n=2):
    result = []
    for i in range(2*n):
        if i-i/2*2 == 0:
            c = "A%d"%(i/2)
            result.append(c)
        else:
            c = "x%d"%(i/2)
            result.append(c)
    return result

Here we test $p_1=x^2$ and $p_2=xy+y^2$. And get Grobner basis $g_1=x^2, g_2=xy+y^2, g_3 = y^3$.

In [300]:
p1 = [(1,2,0)]
p2 = [(1,1,1),(1,0,2)]
polynomials = [p1,p2]
mdeg = 5
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 



[[1.0, 2.0, 0.0]]
[[1.0, 1.0, 1.0], [1.0, 0.0, 2.0]]
[[1.0, 0.0, 3.0]]


Here we test $p_1=xyz+xy$ and $p_2=x^2y^2z+yz$. And get Grobner basis $g_1=x^2y^2z+yz, g_2=xyz+xy, g_3=x^2y^2-2yz, g_4 = yz^2+yz$.

In [301]:
p1 = [(1,1,1,1),(1,1,1,0)]
p2 = [(2,0,1,1),(1,2,2,1)]
polynomials = [p1,p2]
mdeg = 5
nvar = 3
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 

[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.0]]
[]
[[1.0, 2.0, 2.0, 0.0], [-2.0, 0.0, 1.0, 1.0]]
[[1.0, 0.0, 1.0, 2.0], [1.0, 0.0, 1.0, 1.0]]


# Solving Gaussian Quaduature for $n=2$

In [302]:
p1= [(1,1,0,0,0),(1,0,0,1,0),(-2,0,0,0,0)]
p2= [(1,1,1,0,0),(1,0,0,1,1)]
p3= [(1,1,2,0,0),(1,0,0,1,2),(-2.0/3,0,0,0,0)]
p4= [(1,1,3,0,0),(1,0,0,1,3)]
polynomials = [p1,p2,p3,p4]
mdeg = 10
nvar = 4
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),['A0','x0','A1','x1'],nvar)

[[1.0, 1.0, 0.0, 0.0, 0.0], [-1.0, 0.0, 0.0, 0.0, 0.0]]
[]
[[1.0, 0.0, 0.0, 0.0, 2.0], [-0.33333333333333331, 0.0, 0.0, 0.0, 0.0]]
[]
[[1.0, 0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 1.0]]
[[1.0, 0.0, 0.0, 1.0, 0.0], [-3.0, 0.0, 0.0, 0.0, 2.0]]
A0-1.00
x1^2.0-0.33
x0+x1
A1-3.00*x1^2.0


# Test complexity for different variable degree

In [303]:
p1= [(1,1,0,0,0),(1,0,0,1,0),(-2,0,0,0,0)]
p2= [(1,1,1,0,0),(1,0,0,1,1)]
p3= [(1,1,2,0,0),(1,0,0,1,2),(-2.0/3,0,0,0,0)]
p4= [(1,1,3,0,0),(1,0,0,1,3)]
polynomials = [p1,p2,p3,p4]
mdeg = 1000
nvar = 4
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),['A0','x0','A1','x1'],nvar)

A0+A1-2.00
A0x0+A1x1
A0x0^2.0+A1x1^2.0-0.67
A0x0^3.0+A1x1^3.0


In [304]:
%timeit run_sparse(M, mdeg,nvar)

10 loops, best of 3: 129 ms per loop


In [305]:
mdeg = 10
nvar = 4
M=matC_sparse(polynomials,mdeg,nvar)
%timeit run_sparse(M, mdeg,nvar)

10 loops, best of 3: 121 ms per loop


In [306]:
a = np.array([1,2])

In [307]:
np.append(a,1)

array([1, 2, 1])